In [31]:
# %%
# U.S. Flight Delays & Cancellations – ETL for Tableau
# Author: Aniket Patole

import os
import re
import json
import numpy as np
import pandas as pd

# ---- Paths ----
DATA_DIR = "/Data/"   # change to "." if running locally in the repo
RAW_CSV = "../data/Airline_Delay_Cause.csv"
DICT_XLSX = "../data/Download_Column_Definitions.xlsx"

OUT_DIR = "./data"
os.makedirs(OUT_DIR, exist_ok=True)

# Outputs
CLEAN_CSV = os.path.join(OUT_DIR, "cleaned_flight_delays.csv")
AGG_AIRLINE_MONTH = os.path.join(OUT_DIR, "agg_airline_month.csv")
AGG_AIRPORT_MONTH = os.path.join(OUT_DIR, "agg_airport_month.csv")
AGG_CAUSE_LONG = os.path.join(OUT_DIR, "agg_cause_long.csv")
DICT_JSON = os.path.join(OUT_DIR, "column_dictionary.json")


In [32]:
# %%
# 1) LOAD DATA + DICTIONARY
RAW_CSV = "../data/Airline_Delay_Cause.csv"
df = pd.read_csv(RAW_CSV)
print("Raw shape:", df.shape)
display(df.head())

# Dictionary sheet(s) are often multiple tabs; try to read all
try:
    xls = pd.ExcelFile(DICT_XLSX)
    dict_tabs = {sheet: pd.read_excel(DICT_XLSX, sheet_name=sheet) for sheet in xls.sheet_names}
except Exception as e:
    print("Could not read dictionary Excel:", e)
    dict_tabs = {}

# Save dictionary to JSON for reference (best-effort)
dict_payload = {}
for name, tab in dict_tabs.items():
    # Try to infer columns like "Column" and "Description"
    cols_lower = [c.lower() for c in tab.columns]
    col_name = None
    col_desc = None
    for c in tab.columns:
        cl = c.lower()
        if "column" in cl or "field" in cl or "name" in cl:
            if col_name is None:
                col_name = c
        if "description" in cl or "definition" in cl or "meaning" in cl:
            if col_desc is None:
                col_desc = c
    if col_name is not None and col_desc is not None:
        for _, row in tab[[col_name, col_desc]].dropna().iterrows():
            dict_payload[str(row[col_name]).strip()] = str(row[col_desc]).strip()

with open(DICT_JSON, "w") as f:
    json.dump(dict_payload, f, indent=2)

print(f"Saved column dictionary (best-effort) → {DICT_JSON}")


Raw shape: (1951, 21)


,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2025,6,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",90.0,26.0,8.63,3.24,...,0.0,9.72,4.0,0.0,1884.0,561.0,223.0,282.0,0.0,818.0
1,2025,6,9E,Endeavor Air Inc.,ABY,"Albany, GA: Southwest Georgia Regional",5.0,1.0,0.00,0.00,...,0.0,1.00,0.0,0.0,124.0,0.0,0.0,0.0,0.0,124.0
2,2025,6,9E,Endeavor Air Inc.,AEX,"Alexandria, LA: Alexandria International",69.0,23.0,7.21,1.82,...,0.0,5.32,2.0,0.0,1698.0,981.0,54.0,294.0,0.0,369.0
3,2025,6,9E,Endeavor Air Inc.,AGS,"Augusta, GA: Augusta Regional at Bush Field",155.0,43.0,12.78,2.69,...,0.0,17.19,9.0,0.0,2877.0,827.0,198.0,517.0,0.0,1335.0
4,2025,6,9E,Endeavor Air Inc.,ALB,"Albany, NY: Albany International",86.0,29.0,9.32,0.00,...,0.0,15.61,4.0,0.0,1934.0,638.0,0.0,194.0,0.0,1102.0


Could not read dictionary Excel: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.
Saved column dictionary (best-effort) → ./data/column_dictionary.json


<bound method NDFrame.describe of      AIRPORT                                       AIRPORT_NAME  ARR_DEL15  \
0        ABE  Allentown/Bethlehem/Easton, PA: Lehigh Valley ...       26.0   
1        ABY             Albany, GA: Southwest Georgia Regional        1.0   
2        AEX           Alexandria, LA: Alexandria International       23.0   
3        AGS        Augusta, GA: Augusta Regional at Bush Field       43.0   
4        ALB                   Albany, NY: Albany International       29.0   
...      ...                                                ...        ...   
1946     TPA                     Tampa, FL: Tampa International        2.0   
1947     TUL                     Tulsa, OK: Tulsa International       18.0   
1948     TVC                  Traverse City, MI: Cherry Capital        4.0   
1949     TYS                        Knoxville, TN: McGhee Tyson       27.0   
1950     XNA      Fayetteville, AR: Northwest Arkansas National       27.0   

      ARR_FLIGHTS CARRIER  CA

In [33]:
# %%
# 2) STANDARDIZE COLUMNS (UPPERCASE, STRIP), INFER KEY FIELDS

df.columns = [c.strip().upper() for c in df.columns]

# Likely column patterns in BTS On-Time "Delay Cause" dataset
colmap = {
    "YEAR": None,
    "MONTH": None,
    "DAY": None,  # sometimes absent in monthly aggregates
    "CARRIER": None,  # airline code
    "AIRLINE": None,  # sometimes textual airline name
    "AIRPORT": None,  # airport code
    "AIRPORT_NAME": None,  # text name
    "ARR_DEL15": None,     # delayed flights (arrivals delayed >=15 min) count
    "CANCELLED": None,
    "DIVERTED": None,
    "ARR_Flights": None,   # total arriving flights (sometimes ARR_Flights or FLIGHTS)
    "FLIGHTS": None,       # total flights
    "CARRIER_DELAY": None,
    "WEATHER_DELAY": None,
    "NAS_DELAY": None,
    "SECURITY_DELAY": None,
    "LATE_AIRCRAFT_DELAY": None
}

# Helper to set colmap if a matching column exists
def set_if_exists(key, candidates):
    for cand in candidates:
        if cand in df.columns:
            colmap[key] = cand
            return

set_if_exists("YEAR", ["YEAR"])
set_if_exists("MONTH", ["MONTH"])
set_if_exists("DAY", ["DAY_OF_MONTH", "DAY"])
set_if_exists("CARRIER", ["CARRIER", "UNIQUE_CARRIER", "OP_UNIQUE_CARRIER"])
set_if_exists("AIRLINE", ["AIRLINE", "CARRIER_NAME"])
set_if_exists("AIRPORT", ["AIRPORT", "ORIGIN_AIRPORT_ID", "ORIGIN", "AIRPORT_CODE"])
set_if_exists("AIRPORT_NAME", ["AIRPORT_NAME", "ORIGIN_CITY_NAME", "ORIGIN_AIRPORT_NAME"])
set_if_exists("ARR_DEL15", ["ARR_DEL15", "ARR_DEL_15", "ARR_DEL"])
set_if_exists("CANCELLED", ["CANCELLED"])
set_if_exists("DIVERTED", ["DIVERTED"])
set_if_exists("ARR_Flights", ["ARR_FLIGHTS", "ARR_Flights"])
set_if_exists("FLIGHTS", ["FLIGHTS", "TOTAL_FLIGHTS", "CNT_FLIGHTS"])
set_if_exists("CARRIER_DELAY", ["CARRIER_DELAY"])
set_if_exists("WEATHER_DELAY", ["WEATHER_DELAY"])
set_if_exists("NAS_DELAY", ["NAS_DELAY"])
set_if_exists("SECURITY_DELAY", ["SECURITY_DELAY"])
set_if_exists("LATE_AIRCRAFT_DELAY", ["LATE_AIRCRAFT_DELAY"])

print("Detected columns:", json.dumps(colmap, indent=2))


Detected columns: {
  "YEAR": "YEAR",
  "MONTH": "MONTH",
  "DAY": null,
  "CARRIER": "CARRIER",
  "AIRLINE": "CARRIER_NAME",
  "AIRPORT": "AIRPORT",
  "AIRPORT_NAME": "AIRPORT_NAME",
  "ARR_DEL15": "ARR_DEL15",
  "CANCELLED": null,
  "DIVERTED": null,
  "ARR_Flights": "ARR_FLIGHTS",
  "FLIGHTS": null,
  "CARRIER_DELAY": "CARRIER_DELAY",
  "WEATHER_DELAY": "WEATHER_DELAY",
  "NAS_DELAY": "NAS_DELAY",
  "SECURITY_DELAY": "SECURITY_DELAY",
  "LATE_AIRCRAFT_DELAY": "LATE_AIRCRAFT_DELAY"
}


In [34]:
# %%
# 3) BASIC CLEANING

# Keep only relevant columns (don’t error if some are missing)
keep_cols = [c for c in colmap.values() if c is not None]
keep_cols = sorted(set(keep_cols))  # unique
df = df[keep_cols].copy()

# Convert numerics
num_candidates = [
    colmap[k] for k in ["ARR_DEL15", "CANCELLED", "DIVERTED", "ARR_Flights", "FLIGHTS",
                        "CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]
    if colmap[k] is not None
]
for c in num_candidates:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Year/Month to a convenient DATE (use first day of month if DAY missing)
if colmap["YEAR"] and colmap["MONTH"]:
    df["REF_YEAR"] = pd.to_numeric(df[colmap["YEAR"]], errors="coerce")
    df["REF_MONTH"] = pd.to_numeric(df[colmap["MONTH"]], errors="coerce")
    df["REF_DAY"] = 1
    if colmap["DAY"]:
        df["REF_DAY"] = pd.to_numeric(df[colmap["DAY"]], errors="coerce").fillna(1).astype(int)
    df["DATE"] = pd.to_datetime(dict(year=df["REF_YEAR"], month=df["REF_MONTH"], day=df["REF_DAY"]), errors="coerce")
else:
    df["DATE"] = pd.NaT

# Normalize carrier and airport codes to uppercase trimmed strings
if colmap["CARRIER"] and colmap["CARRIER"] in df.columns:
    df["CARRIER_CODE"] = df[colmap["CARRIER"]].astype(str).str.strip().str.upper()
elif colmap["AIRLINE"] and colmap["AIRLINE"] in df.columns:
    # fallback if only AIRLINE textual name is present
    df["CARRIER_CODE"] = df[colmap["AIRLINE"]].astype(str).str.strip().str.upper()
else:
    df["CARRIER_CODE"] = np.nan

if colmap["AIRPORT"] and colmap["AIRPORT"] in df.columns:
    df["AIRPORT_CODE"] = df[colmap["AIRPORT"]].astype(str).str.strip().str.upper()
else:
    df["AIRPORT_CODE"] = np.nan

# Tidy airline/airport names
if colmap["AIRLINE"]:
    df["AIRLINE_NAME"] = df[colmap["AIRLINE"]].astype(str).str.strip()
if colmap["AIRPORT_NAME"]:
    df["AIRPORT_NAME_T"] = df[colmap["AIRPORT_NAME"]].astype(str).str.strip()

print("Clean shape:", df.shape)
display(df.head())


Clean shape: (1951, 21)


,AIRPORT,AIRPORT_NAME,ARR_DEL15,ARR_FLIGHTS,CARRIER,CARRIER_DELAY,CARRIER_NAME,LATE_AIRCRAFT_DELAY,MONTH,NAS_DELAY,...,WEATHER_DELAY,YEAR,REF_YEAR,REF_MONTH,REF_DAY,DATE,CARRIER_CODE,AIRPORT_CODE,AIRLINE_NAME,AIRPORT_NAME_T
0,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",26.0,90.0,9E,561.0,Endeavor Air Inc.,818.0,6,282.0,...,223.0,2025,2025,6,1,2025-06-01,9E,ABE,Endeavor Air Inc.,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ..."
1,ABY,"Albany, GA: Southwest Georgia Regional",1.0,5.0,9E,0.0,Endeavor Air Inc.,124.0,6,0.0,...,0.0,2025,2025,6,1,2025-06-01,9E,ABY,Endeavor Air Inc.,"Albany, GA: Southwest Georgia Regional"
2,AEX,"Alexandria, LA: Alexandria International",23.0,69.0,9E,981.0,Endeavor Air Inc.,369.0,6,294.0,...,54.0,2025,2025,6,1,2025-06-01,9E,AEX,Endeavor Air Inc.,"Alexandria, LA: Alexandria International"
3,AGS,"Augusta, GA: Augusta Regional at Bush Field",43.0,155.0,9E,827.0,Endeavor Air Inc.,1335.0,6,517.0,...,198.0,2025,2025,6,1,2025-06-01,9E,AGS,Endeavor Air Inc.,"Augusta, GA: Augusta Regional at Bush Field"
4,ALB,"Albany, NY: Albany International",29.0,86.0,9E,638.0,Endeavor Air Inc.,1102.0,6,194.0,...,0.0,2025,2025,6,1,2025-06-01,9E,ALB,Endeavor Air Inc.,"Albany, NY: Albany International"


In [35]:
# %%
# 4) KPI DERIVATIONS (robust)

def as_series(colname, default=np.nan):
    """Return a numeric Series for colname if present; else a Series filled with default."""
    if colname and (colname in df.columns):
        return pd.to_numeric(df[colname], errors="coerce")
    return pd.Series(default, index=df.index, dtype="float64")

# Prefer 'FLIGHTS' for total flights; fall back to ARR_FLIGHTS if present
total_flights_col = colmap["FLIGHTS"] or colmap["ARR_Flights"]
if total_flights_col is None:
    print("WARNING: No total flights column found (FLIGHTS/ARR_FLIGHTS missing).")

# Build core series (NEVER scalars)
total_flights = as_series(total_flights_col)
arr_del15     = as_series(colmap["ARR_DEL15"], default=0.0)       # count of delayed flights (>=15m)
cancelled     = as_series(colmap["CANCELLED"], default=np.nan)     # may not exist in this dataset
diverted      = as_series(colmap["DIVERTED"], default=np.nan)      # may not exist in this dataset

# Delay minutes by cause (some datasets have minutes, not counts)
carrier_min   = as_series(colmap["CARRIER_DELAY"], default=0.0).fillna(0)
weather_min   = as_series(colmap["WEATHER_DELAY"], default=0.0).fillna(0)
nas_min       = as_series(colmap["NAS_DELAY"], default=0.0).fillna(0)
security_min  = as_series(colmap["SECURITY_DELAY"], default=0.0).fillna(0)
lateac_min    = as_series(colmap["LATE_AIRCRAFT_DELAY"], default=0.0).fillna(0)

# Fill sensible defaults
total_flights = total_flights.fillna(0)
arr_del15     = arr_del15.fillna(0)

# Derived counts
df["TOTAL_FLIGHTS"]      = total_flights
df["DELAYED_FLIGHTS"]    = arr_del15
df["CANCELLED_FLIGHTS"]  = cancelled.fillna(0)   # if missing in source, this will be 0 (but still a Series)
df["DIVERTED_FLIGHTS"]   = diverted.fillna(0)

# Percent metrics as FRACTIONS (0..1). Safer to format as % later in Tableau/Excel.
# Use np.where to avoid division by zero producing inf.
df["PCT_DELAYED"]   = np.where(df["TOTAL_FLIGHTS"] > 0, df["DELAYED_FLIGHTS"]  / df["TOTAL_FLIGHTS"], np.nan)
df["PCT_CANCELLED"] = np.where(df["TOTAL_FLIGHTS"] > 0, df["CANCELLED_FLIGHTS"]/ df["TOTAL_FLIGHTS"], np.nan)
df["PCT_DIVERTED"]  = np.where(df["TOTAL_FLIGHTS"] > 0, df["DIVERTED_FLIGHTS"] / df["TOTAL_FLIGHTS"], np.nan)

# Total delay minutes across causes
df["TOTAL_DELAY_MIN"] = (carrier_min + weather_min + nas_min + security_min + lateac_min)

# Helpful debug print
print("Detected columns:",
      {"FLIGHTS": total_flights_col,
       "ARR_DEL15": colmap["ARR_DEL15"],
       "CANCELLED": colmap["CANCELLED"],
       "DIVERTED": colmap["DIVERTED"]})

display(df[[
    "DATE","CARRIER_CODE","AIRPORT_CODE","TOTAL_FLIGHTS",
    "DELAYED_FLIGHTS","CANCELLED_FLIGHTS","DIVERTED_FLIGHTS",
    "PCT_DELAYED","PCT_CANCELLED","PCT_DIVERTED","TOTAL_DELAY_MIN"
]].head())


Detected columns: {'FLIGHTS': 'ARR_FLIGHTS', 'ARR_DEL15': 'ARR_DEL15', 'CANCELLED': None, 'DIVERTED': None}


,DATE,CARRIER_CODE,AIRPORT_CODE,TOTAL_FLIGHTS,DELAYED_FLIGHTS,CANCELLED_FLIGHTS,DIVERTED_FLIGHTS,PCT_DELAYED,PCT_CANCELLED,PCT_DIVERTED,TOTAL_DELAY_MIN
0,2025-06-01,9E,ABE,90.0,26.0,0.0,0.0,0.288889,0.0,0.0,1884.0
1,2025-06-01,9E,ABY,5.0,1.0,0.0,0.0,0.200000,0.0,0.0,124.0
2,2025-06-01,9E,AEX,69.0,23.0,0.0,0.0,0.333333,0.0,0.0,1698.0
3,2025-06-01,9E,AGS,155.0,43.0,0.0,0.0,0.277419,0.0,0.0,2877.0
4,2025-06-01,9E,ALB,86.0,29.0,0.0,0.0,0.337209,0.0,0.0,1934.0


In [36]:
# %%
# 5) AGGREGATIONS FOR TABLEAU

# Airline x Month level
grp_airline = df.groupby([pd.Grouper(key="DATE", freq="MS"), "CARRIER_CODE"], dropna=False).agg({
    "TOTAL_FLIGHTS": "sum",
    "DELAYED_FLIGHTS": "sum",
    "CANCELLED_FLIGHTS": "sum",
    "DIVERTED_FLIGHTS": "sum",
    "CARRIER_DELAY": "sum",
    "WEATHER_DELAY": "sum",
    "NAS_DELAY": "sum",
    "SECURITY_DELAY": "sum",
    "LATE_AIRCRAFT_DELAY": "sum",
}).reset_index()

for col_n, col_d in [
    ("PCT_DELAYED", ("DELAYED_FLIGHTS", "TOTAL_FLIGHTS")),
    ("PCT_CANCELLED", ("CANCELLED_FLIGHTS", "TOTAL_FLIGHTS")),
    ("PCT_DIVERTED", ("DIVERTED_FLIGHTS", "TOTAL_FLIGHTS"))
]:
    n, d = col_d
    grp_airline[col_n] = np.where(grp_airline[d] > 0, grp_airline[n] / grp_airline[d] * 100, np.nan)

# Airport x Month level
grp_airport = df.groupby([pd.Grouper(key="DATE", freq="MS"), "AIRPORT_CODE"], dropna=False).agg({
    "TOTAL_FLIGHTS": "sum",
    "DELAYED_FLIGHTS": "sum",
    "CANCELLED_FLIGHTS": "sum",
    "DIVERTED_FLIGHTS": "sum",
    "CARRIER_DELAY": "sum",
    "WEATHER_DELAY": "sum",
    "NAS_DELAY": "sum",
    "SECURITY_DELAY": "sum",
    "LATE_AIRCRAFT_DELAY": "sum",
}).reset_index()

for col_n, col_d in [
    ("PCT_DELAYED", ("DELAYED_FLIGHTS", "TOTAL_FLIGHTS")),
    ("PCT_CANCELLED", ("CANCELLED_FLIGHTS", "TOTAL_FLIGHTS")),
    ("PCT_DIVERTED", ("DIVERTED_FLIGHTS", "TOTAL_FLIGHTS"))
]:
    n, d = col_d
    grp_airport[col_n] = np.where(grp_airport[d] > 0, grp_airport[n] / grp_airport[d] * 100, np.nan)

# Long format for delay causes (stacked bars)
cause_cols_present = [c for c in ["CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"] if c in df.columns]
cause_long = (df.melt(
    id_vars=["DATE","CARRIER_CODE","AIRPORT_CODE"],
    value_vars=cause_cols_present,
    var_name="DELAY_CAUSE",
    value_name="DELAY_MINUTES"
).groupby(["DATE","CARRIER_CODE","AIRPORT_CODE","DELAY_CAUSE"], dropna=False)["DELAY_MINUTES"].sum().reset_index())

print("Agg airline shape:", grp_airline.shape)
print("Agg airport shape:", grp_airport.shape)
print("Cause long shape:", cause_long.shape)

display(grp_airline.head())
display(grp_airport.head())
display(cause_long.head())


Agg airline shape: (20, 14)
Agg airport shape: (360, 14)
Cause long shape: (9755, 5)


,DATE,CARRIER_CODE,TOTAL_FLIGHTS,DELAYED_FLIGHTS,CANCELLED_FLIGHTS,DIVERTED_FLIGHTS,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,PCT_DELAYED,PCT_CANCELLED,PCT_DIVERTED
0,2025-06-01,9E,20674.0,5796.0,0.0,0.0,151726.0,29861.0,124838.0,33.0,225262.0,28.035213,0.0,0.0
1,2025-06-01,AA,85419.0,27213.0,0.0,0.0,715003.0,192717.0,373066.0,2082.0,1127962.0,31.858252,0.0,0.0
2,2025-06-01,AS,22576.0,6773.0,0.0,0.0,88720.0,9757.0,94096.0,2462.0,145839.0,30.000886,0.0,0.0
3,2025-06-01,B6,18953.0,5299.0,0.0,0.0,110963.0,13718.0,125761.0,1185.0,167565.0,27.958635,0.0,0.0
4,2025-06-01,C5,7288.0,2301.0,0.0,0.0,94157.0,30055.0,38296.0,0.0,156239.0,31.572448,0.0,0.0


,DATE,AIRPORT_CODE,TOTAL_FLIGHTS,DELAYED_FLIGHTS,CANCELLED_FLIGHTS,DIVERTED_FLIGHTS,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,PCT_DELAYED,PCT_CANCELLED,PCT_DIVERTED
0,2025-06-01,ABE,408.0,135.0,0.0,0.0,2544.0,1682.0,1528.0,0.0,4997.0,33.088235,0.0,0.0
1,2025-06-01,ABI,146.0,48.0,0.0,0.0,361.0,278.0,315.0,0.0,1388.0,32.876712,0.0,0.0
2,2025-06-01,ABQ,2414.0,685.0,0.0,0.0,15754.0,1158.0,5182.0,0.0,21446.0,28.376139,0.0,0.0
3,2025-06-01,ABR,60.0,4.0,0.0,0.0,877.0,58.0,2.0,0.0,0.0,6.666667,0.0,0.0
4,2025-06-01,ABY,86.0,34.0,0.0,0.0,2634.0,1178.0,972.0,0.0,124.0,39.534884,0.0,0.0


,DATE,CARRIER_CODE,AIRPORT_CODE,DELAY_CAUSE,DELAY_MINUTES
0,2025-06-01,9E,ABE,CARRIER_DELAY,561.0
1,2025-06-01,9E,ABE,LATE_AIRCRAFT_DELAY,818.0
2,2025-06-01,9E,ABE,NAS_DELAY,282.0
3,2025-06-01,9E,ABE,SECURITY_DELAY,0.0
4,2025-06-01,9E,ABE,WEATHER_DELAY,223.0


In [37]:
# %%
# 6) SAVE OUTPUTS FOR TABLEAU

# Cleaned row-level (or original grain) with engineered fields
df_out_cols = ["DATE", "CARRIER_CODE", "AIRPORT_CODE", "TOTAL_FLIGHTS",
               "DELAYED_FLIGHTS","CANCELLED_FLIGHTS","DIVERTED_FLIGHTS",
               "PCT_DELAYED","PCT_CANCELLED","PCT_DIVERTED",
               "CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY",
               "TOTAL_DELAY_MIN"]
df_export = df[df_out_cols].copy()

df_export.to_csv(CLEAN_CSV, index=False)
grp_airline.to_csv(AGG_AIRLINE_MONTH, index=False)
grp_airport.to_csv(AGG_AIRPORT_MONTH, index=False)
cause_long.to_csv(AGG_CAUSE_LONG, index=False)

print("Saved:")
print(" -", CLEAN_CSV)
print(" -", AGG_AIRLINE_MONTH)
print(" -", AGG_AIRPORT_MONTH)
print(" -", AGG_CAUSE_LONG)


Saved:
 - ./data/cleaned_flight_delays.csv
 - ./data/agg_airline_month.csv
 - ./data/agg_airport_month.csv
 - ./data/agg_cause_long.csv


In [38]:
# %%
# 7) QUICK DATA HEALTH CHECKS (OPTIONAL)

summary = {
    "date_min": str(df["DATE"].min()),
    "date_max": str(df["DATE"].max()),
    "airlines": int(df["CARRIER_CODE"].nunique()),
    "airports": int(df["AIRPORT_CODE"].nunique()),
    "rows": int(df.shape[0]),
    "total_flights_sum": float(df["TOTAL_FLIGHTS"].sum()) if "TOTAL_FLIGHTS" in df else None,
    "pct_delayed_mean": float(np.nanmean(df["PCT_DELAYED"])) if "PCT_DELAYED" in df else None,
}

print("Dataset summary:", json.dumps(summary, indent=2))


Dataset summary: {
  "date_min": "2025-06-01 00:00:00",
  "date_max": "2025-06-01 00:00:00",
  "airlines": 20,
  "airports": 360,
  "rows": 1951,
  "total_flights_sum": 674179.0,
  "pct_delayed_mean": 0.2866839664677477
}


In [39]:
# === Paths ===
OUT_DIR = "../data"  # adjust if needed
CLEAN_CSV = f"{OUT_DIR}/cleaned_flight_delays.csv"
AGG_AIRLINE_MONTH = f"{OUT_DIR}/agg_airline_month.csv"
AGG_AIRPORT_MONTH = f"{OUT_DIR}/agg_airport_month.csv"
AGG_CAUSE_LONG = f"{OUT_DIR}/agg_cause_long.csv"

CLEAN_XLSX = f"{OUT_DIR}/cleaned_flight_delays.xlsx"
AGG_AIRLINE_XLSX = f"{OUT_DIR}/agg_airline_month.xlsx"
AGG_AIRPORT_XLSX = f"{OUT_DIR}/agg_airport_month.xlsx"
AGG_CAUSE_XLSX = f"{OUT_DIR}/agg_cause_long.xlsx"

# --- 1) Save fresh CSVs (UTF-8, Unix newlines) ---
df_export.to_csv(CLEAN_CSV, index=False, encoding="utf-8", lineterminator="\n")
grp_airline.to_csv(AGG_AIRLINE_MONTH, index=False, encoding="utf-8", lineterminator="\n")
grp_airport.to_csv(AGG_AIRPORT_MONTH, index=False, encoding="utf-8", lineterminator="\n")
cause_long.to_csv(AGG_CAUSE_LONG, index=False, encoding="utf-8", lineterminator="\n")

# --- 2) Save XLSX versions (preferred for Tableau on macOS) ---
with pd.ExcelWriter(CLEAN_XLSX, engine="xlsxwriter", datetime_format="yyyy-mm-dd") as xw:
    df_export.to_excel(xw, index=False, sheet_name="data")

with pd.ExcelWriter(AGG_AIRLINE_XLSX, engine="xlsxwriter", datetime_format="yyyy-mm-dd") as xw:
    grp_airline.to_excel(xw, index=False, sheet_name="airline_month")

with pd.ExcelWriter(AGG_AIRPORT_XLSX, engine="xlsxwriter", datetime_format="yyyy-mm-dd") as xw:
    grp_airport.to_excel(xw, index=False, sheet_name="airport_month")

with pd.ExcelWriter(AGG_CAUSE_XLSX, engine="xlsxwriter", datetime_format="yyyy-mm-dd") as xw:
    cause_long.to_excel(xw, index=False, sheet_name="cause_long")

print("Saved files:\n",
      CLEAN_CSV, "\n", AGG_AIRLINE_MONTH, "\n", AGG_AIRPORT_MONTH, "\n", AGG_CAUSE_LONG, "\n",
      CLEAN_XLSX, "\n", AGG_AIRLINE_XLSX, "\n", AGG_AIRPORT_XLSX, "\n", AGG_CAUSE_XLSX)


Saved files:
 ../data/cleaned_flight_delays.csv 
 ../data/agg_airline_month.csv 
 ../data/agg_airport_month.csv 
 ../data/agg_cause_long.csv 
 ../data/cleaned_flight_delays.xlsx 
 ../data/agg_airline_month.xlsx 
 ../data/agg_airport_month.xlsx 
 ../data/agg_cause_long.xlsx


In [42]:
# Assuming DATE is a datetime column
df["MONTH"] = pd.to_datetime(df["DATE"]).dt.to_period("M").dt.to_timestamp()

grp_airline = (
    df.groupby(["MONTH", "CARRIER_CODE"], as_index=False)
      .agg({
          "TOTAL_FLIGHTS": "sum",
          "DELAYED_FLIGHTS": "sum",
          "CANCELLED_FLIGHTS": "sum",
          "DIVERTED_FLIGHTS": "sum",
          "CARRIER_DELAY": "sum",
          "WEATHER_DELAY": "sum",
          "NAS_DELAY": "sum",
          "SECURITY_DELAY": "sum",
          "LATE_AIRCRAFT_DELAY": "sum",
          "TOTAL_DELAY_MIN": "sum"
      })
)

# Recompute percents at the aggregated level
grp_airline["PCT_DELAYED"]   = grp_airline["DELAYED_FLIGHTS"] / grp_airline["TOTAL_FLIGHTS"]
grp_airline["PCT_CANCELLED"] = grp_airline["CANCELLED_FLIGHTS"] / grp_airline["TOTAL_FLIGHTS"]
grp_airline["PCT_DIVERTED"]  = grp_airline["DIVERTED_FLIGHTS"] / grp_airline["TOTAL_FLIGHTS"]

# Export
grp_airline.to_excel("agg_airline_month.xlsx", index=False)
